In [2]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 438.6 kB/s eta 0:00:00


In [1]:
!pip uninstall triton

Found existing installation: triton 2.3.0
Uninstalling triton-2.3.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/triton-2.3.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/triton/*
Proceed (Y/n)? Y
  Successfully uninstalled triton-2.3.0


In [3]:
import torch
from transformers import AutoTokenizer, AutoModel, BertConfig

# Cargamos el modelo DNABERT-6 con su tokenizer y configuración, los demás modelos están en las respectivas rutas (cambiando 6 por el tamaño de los k-meros)
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNA_bert_6", trust_remote_code=True)
config = BertConfig.from_pretrained("zhihan1996/DNA_bert_6")
model = AutoModel.from_pretrained("zhihan1996/DNA_bert_6", trust_remote_code=True, config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

configuration_bert.py:   0%|          | 0.00/807 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

dnabert_layer.py:   0%|          | 0.00/5.44k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNA_bert_6:
- dnabert_layer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/359M [00:00<?, ?B/s]

In [4]:
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(4101, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [5]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(4101, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [7]:
# Esta función nos permitirá tokenizar correctameante las secuencias con los tokenizer de los modelos, para DNABERT-S
# Con los modelos DNABERT-2-117M y DNABERT-S, se debe adaptar el método para obtener la tokenización correspondiente
def generate_kmers(sequence, k=6):
    # Inicializamos una lista para almacenar los k-meros
    kmers = []

    # Iteramos sobre la secuencia para extraer los k-meros
    # Dejamos de iterar cuando no haya suficiente caracteres para formar un nuevo k-mero
    for i in range(len(sequence) - k + 1):
        # Extraemos los k-meros comenzando en la posición i
        kmer = sequence[i:i+k]

        # Añadimos el k-mero a la lista
        kmers.append(kmer)

    # Juntamos todos los k-meros en una cadena única con espacios entre ellos
    return ' '.join(kmers)

In [17]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
dna = generate_kmers(dna*8 + 'A')
inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"].to('cuda')
hidden_states = model(inputs)[0] # [1, sequence_length, 768]

In [18]:
# embedding con mean pooling
embedding_mean = torch.mean(hidden_states[0], dim=0)
print(embedding_mean.shape) # 768

torch.Size([768])


In [16]:
embedding_mean

tensor([-3.8977e-01,  5.1028e-01, -3.7793e-01, -7.0666e-01,  2.2994e-01,
        -2.6080e-01,  2.5784e-01, -6.3152e-01, -4.7499e-01,  2.9980e-01,
        -6.7917e-01, -4.0811e-01, -3.2591e-01,  5.9283e-01, -8.6556e-02,
         5.7544e-02, -1.2260e-01, -1.1853e-01, -1.0117e+00,  3.8885e-01,
        -7.0051e-01, -4.9564e-01, -7.4965e-01, -1.8443e-02, -9.1897e-01,
        -3.7846e-02,  4.6934e-01,  7.1451e-01, -1.8060e-01, -6.7759e-01,
         1.0839e-01, -5.7583e-02,  9.9869e-03, -3.5877e-01,  9.0351e-01,
        -3.8889e-01,  2.3583e-01, -5.0925e-01, -4.7764e-01, -1.5694e-02,
         1.3964e-02, -6.2870e-01, -4.6239e-01, -2.2002e-01, -3.6132e-01,
        -3.6664e-02, -1.7315e-01, -1.2288e+00,  7.7714e-02, -1.0559e+00,
        -7.6506e-01, -3.5456e-02,  1.4700e-01, -1.0094e-01, -4.6086e-02,
         1.2091e-01, -7.0113e-02, -1.4681e-01,  1.9034e-01, -5.1465e-01,
        -8.6983e-01,  2.5379e-01,  9.2832e-02, -8.8690e-02,  4.8656e-01,
         9.7038e-01, -1.3238e-01, -7.4867e-01,  1.8

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
df = pd.read_parquet('drive/MyDrive/TFM/genomic_sequences.parquet')

In [10]:
from pathlib import Path
longitud = df.shape[0]
for i in range(longitud):
  key = df.iloc[i]['gene_id']
  ruta = Path('drive/MyDrive/TFM/Code/embeddings/dnabert-6/'+str(key)+'_0.pt')
  existe = ruta.exists()
  if not existe:
    break
  else:
    print('gene', i, 'stored')

gene 0 stored
gene 1 stored
gene 2 stored
gene 3 stored
gene 4 stored
gene 5 stored
gene 6 stored
gene 7 stored
gene 8 stored
gene 9 stored
gene 10 stored
gene 11 stored
gene 12 stored
gene 13 stored
gene 14 stored
gene 15 stored
gene 16 stored
gene 17 stored
gene 18 stored
gene 19 stored
gene 20 stored
gene 21 stored
gene 22 stored
gene 23 stored
gene 24 stored
gene 25 stored
gene 26 stored
gene 27 stored
gene 28 stored
gene 29 stored
gene 30 stored
gene 31 stored
gene 32 stored
gene 33 stored
gene 34 stored
gene 35 stored
gene 36 stored
gene 37 stored
gene 38 stored
gene 39 stored
gene 40 stored
gene 41 stored
gene 42 stored
gene 43 stored
gene 44 stored
gene 45 stored
gene 46 stored
gene 47 stored
gene 48 stored
gene 49 stored
gene 50 stored
gene 51 stored
gene 52 stored
gene 53 stored
gene 54 stored
gene 55 stored
gene 56 stored
gene 57 stored
gene 58 stored
gene 59 stored
gene 60 stored
gene 61 stored
gene 62 stored
gene 63 stored
gene 64 stored
gene 65 stored
gene 66 stored
gene 

In [ ]:
# Importamos la librería Path que permite saber si una ruta existe
# from pathlib import Path

# Importamos la librería Garbage Collector para borrado de memoria en la sesión a través de las iteraciones y que la sesión no termine
import gc

# Número de genes para los que estamos sacando los embeddings
longitud = df.shape[0]


# Bucle que itera sobre los distintos genes generando y guardando archivos con los embeddings
for i in range(longitud):

  # Identificador del gen i-ésimo
  key = df.iloc[i]['gene_id']

  # ruta = Path('drive/MyDrive/TFM/Code/embeddings/DNABERT-S/'+str(key)+'.pt')
  # existe = ruta.exists()
  #if not existe:

  value = df.iloc[i]['sequence']  # secuencia del gen
  value = generate_kmers(value)
  inputs = tokenizer(value, return_tensors = 'pt')["input_ids"].to('cuda')  # tokenización de la secuencia para poder introducirlo al modelo
  inputs = inputs.to('cuda')
  cont = inputs.shape[1]  # longitud de la tokenización

  num_chunks = (cont + 511) // 512  # +511 y no 512 para redondear hacia arriba
  print(i, key, num_chunks)
  total_chunks = torch.chunk(inputs, num_chunks, dim=1)  
  # distribuimos los valores de la tokenización para que no superen la longitud 512, que es el máximo que toma el modelo


  # Creamos agrupaciones de los embeddings para que se guarden en grupos de 10 en 10 e ir tomando la media de los embeddings
  grouped_chunks = []
  for idx in range(0, len(total_chunks), 10):
    remaining_elements = len(total_chunks) - idx
    if 10 < remaining_elements < 20:
        next_chunk_size = remaining_elements // 2

        # Esto lo he hecho para casos en los que tengamos casos con número de chunks como 12 o 21, donde esos chunks que no entran en decenas tendrían mucho más peso y
        # de esta manera se evitan estos casos, partiendo en esos casos tomando grupos de 6-6 y 6-5 (respectivamente)
        grouped_chunks.append(total_chunks[idx:idx+next_chunk_size])
        grouped_chunks.append(total_chunks[idx+next_chunk_size:])
        break
    else:
        grouped_chunks.append(total_chunks[idx:idx+10])

  # Al borrar el acceso a la variable se permite su borrado en memoria
  del total_chunks
  gc.collect()

  # Iteramos por cada bloque de chunks generando los archivos, los genes que tengan varios archivos luego tendremos que promediarlos una vez terminado el proceso
  for j, chunks in enumerate(grouped_chunks):
    hidden_states = []

    # Aplicamos el modelo a cada uno de los chunks de las listas de chunks
    for chunk in chunks:
      hidden_states += [model(chunk)[0]] # [1, sequence_length, 768]
      gc.collect()

    # Creamos el embedding del chunk a partir de la media de los embeddings de la lista del chunk
    embedding_mean = []
    for hidden_state in hidden_states:
      embedding_mean += [torch.mean(hidden_state[0], dim=0)]
      gc.collect()
    stacked_tensors = torch.stack(embedding_mean)
    embedding_mean = torch.mean(stacked_tensors, dim=0)

    # Guardamos el archivo correspondiente
    key = df.iloc[i]['gene_id']
    torch.save(embedding_mean, 'drive/MyDrive/TFM/Code/embeddings/dnabert-6/'+str(key)+'_'+str(j)+'.pt')
    print(i, key, j, 'stored')

    # Al borrar el acceso a las variables se permite su borrado en memoria
    del hidden_states
    del embedding_mean
    del stacked_tensors

    gc.collect()

Se han truncado las últimas 5000 líneas del flujo de salida.
3340 5090 21 stored
3340 5090 22 stored
3340 5090 23 stored
3340 5090 24 stored
3340 5090 25 stored
3340 5090 26 stored
3340 5090 27 stored
3340 5090 28 stored
3340 5090 29 stored
3340 5090 30 stored
3340 5090 31 stored
3340 5090 32 stored
3340 5090 33 stored
3340 5090 34 stored
3340 5090 35 stored
3340 5090 36 stored
3340 5090 37 stored
3340 5090 38 stored
3340 5090 39 stored
3340 5090 40 stored
3340 5090 41 stored
3340 5090 42 stored
3341 5091 215
3341 5091 0 stored
3341 5091 1 stored
3341 5091 2 stored
3341 5091 3 stored
3341 5091 4 stored
3341 5091 5 stored
3341 5091 6 stored
3341 5091 7 stored
3341 5091 8 stored
3341 5091 9 stored
3341 5091 10 stored
3341 5091 11 stored
3341 5091 12 stored
3341 5091 13 stored
3341 5091 14 stored
3341 5091 15 stored
3341 5091 16 stored
3341 5091 17 stored
3341 5091 18 stored
3341 5091 19 stored
3341 5091 20 stored
3341 5091 21 stored
3342 5092 13
3342 5092 0 stored
3342 5092 1 stored
3343